In [1]:
# Install Java, Spark, and Findspark
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-ba

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETLProject").getOrCreate()

In [19]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame 
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), 
                                             inferSchema=True, sep='\t', timestampFormat="yyyy-MM-dd")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

## How many rows are in the dataset?

In [20]:
# Count the number of entries in the dataset
print(f"There are {df.count()} rows in the dataset")

There are 1785997 rows in the dataset


## Transform DataFrame to fit review_id table

In [21]:
review_df = df.select(["review_id", "customer_id","product_id", "product_parent", "review_date"])
review_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809|2015-08-31 00:00:00|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175|2015-08-31 00:00:00|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668|2015-08-31 00:00:00|
|R127K9NTSXA2YH|   52495923|B00GOOSV98|      23143350|2015-08-31 00:00:00|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511|2015-08-31 00:00:00|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615|2015-08-31 00:00:00|
|R2F0POU5K6F73F|   17521011|B008XHCLFO|      24234603|2015-08-31 00:00:00|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517|2015-08-31 00:00:00|
| R3GZTM72WA2QH|     224068|B009EPWJLA|     435241890|2015-08-31 00:00:00|
| RNQOY62705W1K|   48467989|B0000AV7GB|     256572651|2015-08-31 00:00:00|
|R1VTIA3JTYBY02|     1065

In [22]:
# Show the schema to confirm the column type
review_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [24]:
# Drop the hh:mm:ss from the timestamp

# Import date time functions
from pyspark.sql.functions import date_format

review_df = review_df.withColumn("review_date", date_format(review_df['review_date'], 'yyyy-MM-dd'))
review_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809| 2015-08-31|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175| 2015-08-31|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668| 2015-08-31|
|R127K9NTSXA2YH|   52495923|B00GOOSV98|      23143350| 2015-08-31|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511| 2015-08-31|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615| 2015-08-31|
|R2F0POU5K6F73F|   17521011|B008XHCLFO|      24234603| 2015-08-31|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517| 2015-08-31|
| R3GZTM72WA2QH|     224068|B009EPWJLA|     435241890| 2015-08-31|
| RNQOY62705W1K|   48467989|B0000AV7GB|     256572651| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [25]:
# Drop any null values
review_df.dropna()

DataFrame[review_id: string, customer_id: int, product_id: string, product_parent: int, review_date: string]

In [26]:
# Check to see if the number of rows matches the dataset
print(f"There are {review_df.count()} rows in the data frame")

There are 1785997 rows in the data frame


## Transform Dataframe to match products table

In [27]:
products_df = df.select(["product_id", "product_title"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001CXYMFS|Thrustmaster T-Fl...|
|B00M920ND6|Tonsee 6 buttons ...|
|B0029CSOD2|Hidden Mysteries:...|
|B00GOOSV98|GelTabz Performan...|
|B00Y074JOM|Zero Suit Samus a...|
|B002UBI6W6|Psyclone Recharge...|
|B008XHCLFO|Protection for yo...|
|B00BRA9R6A|   Nerf 3DS XL Armor|
|B009EPWJLA|One Piece: Pirate...|
|B0000AV7GB|Playstation 2 Dan...|
|B00008KTNN|Metal Arms: Glitc...|
|B000A3IA0Y|72 Pin Connector ...|
|B0053BQN34|uDraw Gametablet ...|
|B002I0HIMI|NBA 2K12(Covers M...|
|B0081EH12M|New Trigger Grips...|
|B005FMLZQQ|Xbox 360 Media Re...|
|B002BSA388|Super Mario Galaxy 2|
|B00BUSLSAC|Nintendo 3DS XL -...|
|B00KWF38AW|Captain Toad:  Tr...|
|B00BRQJYA8|Lego Batman 2: DC...|
+----------+--------------------+
only showing top 20 rows



In [28]:
# Check the columns to match the schema
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [29]:
# Check to see if the number of rows matches the dataset
print(f"There are {products_df.count()} rows in the data frame")

There are 1785997 rows in the data frame


## Transform the dataframe to match the customers table

In [30]:
customers_df = df.select(["customer_id"])
customers_df.show()

+-----------+
|customer_id|
+-----------+
|   12039526|
|    9636577|
|    2331478|
|   52495923|
|   14533949|
|    2377552|
|   17521011|
|   19676307|
|     224068|
|   48467989|
|     106569|
|   48269642|
|   52738710|
|   10556786|
|    2963837|
|   23092109|
|   23091728|
|   10712640|
|   17455376|
|   14754850|
+-----------+
only showing top 20 rows



In [31]:
customers_df = customers_df.groupBy('customer_id').agg({"customer_id": "count"})
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   48670265|             1|
|   49103216|             2|
|    1131200|             1|
|   43076447|             2|
|   46261368|             1|
|    4883305|             5|
|   41192649|             1|
|   40985731|             7|
|   10437900|             2|
|   22245671|             1|
|    2574873|             1|
|    4696154|             1|
|    5621202|             1|
|    5871933|             2|
|   44089812|             1|
|    2845910|             1|
|    5274369|             1|
|   39069693|             2|
|     137793|             1|
|   31914942|             3|
+-----------+--------------+
only showing top 20 rows



## Transform dataset to match the vine table

In [32]:
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RTIS3L2M1F5SM|          5|            0|          0|   N|
| R1ZV7R40OLHKD|          5|            0|          0|   N|
|R3BH071QLH8QMC|          1|            0|          1|   N|
|R127K9NTSXA2YH|          3|            0|          0|   N|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|
|R2F0POU5K6F73F|          5|            0|          0|   N|
|R3VNR804HYSMR6|          5|            0|          0|   N|
| R3GZTM72WA2QH|          5|            0|          0|   N|
| RNQOY62705W1K|          4|            0|          0|   N|
|R1VTIA3JTYBY02|          5|            0|          0|   N|
|R29DOU8791QZL8|          1|            0|          0|   N|
|R15DUT1VIJ9RJZ|          2|            0|          0|   N|
|R3IMF2MQ3OU9ZM|          4|            

In [33]:
# Check to see if the number of rows matches the dataset
print(f"There are {vine_df.count()} rows in the data frame")

There are 1785997 rows in the data frame


## Setup the RDS

In [38]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.ckkgxbdiidvr.us-east-2.rds.amazonaws.com:5432/big_data_hw_db"
config = {"user":"root", 
          "password": "sKfjDP4s5MTHbdi", 
          "driver":"org.postgresql.Driver"}

In [39]:
# Write DataFrame to review_id table in RDS

review_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write DataFrame to products table in RDS

products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to customers table in RDS

customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to vine table in RDS

vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)